<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%206%20-%20Aprendizaje%20por%20Transferencia%20y%20puesta%20a%20punto%20de%20parámetros%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

In [ ]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [ ]:
# !wget --no-check-certificate \
#     https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
#     -O ./cats_and_dogs_filtered.zip

## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.8.2'

### Descomprimir el dataset de Perros vs Gatos

In [2]:
# dataset_path = "./cats_and_dogs_filtered.zip"
# zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
# zip_object.extractall("./")
# zip_object.close()

### Configurar las rutas al dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset_path_new = "/content/drive/Othercomputers/Mi portátil/02. Programacion/Tensorflow 2.0 - Udemy/datasets/adultos niños"

In [5]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "test")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [6]:
IMG_SHAPE = (128, 128, 3)

In [7]:
# Aquí descargo el modelo base

# include_top=False --> es para eliminar el final de al red, la cabecera, de manera que se pueda hacer el fine tunning

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

In [8]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

### Congelar el modelo base

In [9]:
base_model.trainable = False

# Esto fija los pesas aprendidos antes y permite añadir la cabecera

### Definir la cabecera personalizada para nuestra red neuronal

In [10]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [11]:
#capa para achicar... promediar, los 4x4, en los 1280

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [12]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [13]:
#ultima capaca clasificadora

prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [14]:
# acá se usa Model, par que una la dos redes que he construido

model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [15]:
model.summary()

# NOTESE que al final de imprsión aparece lo congelado: Non-trainable params: 2,257,984

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

### Compilar el modelo

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(224,224), batch_size=128, class_mode="binary")

Found 680 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(224,224), batch_size=128, class_mode="binary")

Found 120 images belonging to 2 classes.


### Entrenar el modelo 

In [ ]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

### Evaluar el modelo de aprendizaje por transferencia

In [ ]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [16]:
base_model.trainable = True

In [17]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [18]:
fine_tune_at = 100

In [19]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [20]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### Puesta a punto

In [25]:
model.fit_generator(train_generator,  
                    epochs=8, 
                    validation_data=valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/8
6/6 [==============================] - 14s 1s/step - loss: 0.5754 - accuracy: 0.6824 - val_loss: 0.7342 - val_accuracy: 0.5917
Epoch 2/8
6/6 [==============================] - 4s 629ms/step - loss: 0.2431 - accuracy: 0.9529 - val_loss: 0.5035 - val_accuracy: 0.7500
Epoch 3/8
6/6 [==============================] - 4s 676ms/step - loss: 0.1305 - accuracy: 0.9882 - val_loss: 0.4537 - val_accuracy: 0.8000
Epoch 4/8
6/6 [==============================] - 4s 636ms/step - loss: 0.0718 - accuracy: 1.0000 - val_loss: 0.4237 - val_accuracy: 0.8333
Epoch 5/8
6/6 [==============================] - 4s 633ms/step - loss: 0.0419 - accuracy: 0.9985 - val_loss: 0.4275 - val_accuracy: 0.8167
Epoch 6/8
6/6 [==============================] - 4s 729ms/step - loss: 0.0270 - accuracy: 1.0000 - val_loss: 0.4199 - val_accuracy: 0.8333
Epoch 7/8
6/6 [==============================] - 4s 626ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.4261 - val_accuracy: 0.8417
Epoch 8/8
6/6 [==============

### Evaluar el modelo re calibrado

In [26]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


In [27]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.8083333373069763
